CHATBOT

In [3]:
import unidecode
import pickle
import pandas as pd
from nltk.tokenize import RegexpTokenizer  
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity


def get_max5(arr):    
    '''
    Get the 5 most relevant answers
    '''
    ixarr = []

    for index, el in enumerate(arr):
        ixarr.append((el, index))
    ixarr.sort(reverse=True)
    
    max_val=max(ixarr[0])

    ixs = []

    for i in ixarr[:5]:
        ixs.append(i)

    return ixs[::-1]



def process(input_text):
    liste_inter=[]
    liste_finale=[]    

    for word in input_text:        
        unaccented_string = unidecode.unidecode(word)
        tokenizer = RegexpTokenizer(r'\w+')
        stemmer = SnowballStemmer('french')
        stop_words = stopwords.words('french')
        tokens = tokenizer.tokenize(unaccented_string)
        tokens = [x for x in tokens if not x in stop_words]
        tokens_stemmed = [stemmer.stem(x) for x in tokens]
        liste_inter.append(tokens_stemmed)

    for elem in liste_inter:
        x=" ".join(elem)
        liste_finale.append(x)
    
    return liste_finale


def get_cos_sims (input_test):
    
    with open('vecto.pkl', 'rb') as file:
        vectorizer = pickle.load(file)
        
    data = pd.read_csv('septembre_df.csv', index_col=0)
    data=data[data.Question != ('Bonjour'or'hello'or'yo'or'salut')]
    x_mix=list(data['Question'].values)
    X_mix=process(x_mix)
    clean_X=vectorizer.fit_transform(X_mix) 
    
    
    usr2=process([input_test])
    tfidf_question = vectorizer.transform(usr2)
    
    cos_sims = []
    
    for i_question in range(len(X_mix)):  
        question_in_bank = clean_X[i_question, :]
        sims = cosine_similarity(tfidf_question, question_in_bank)
        cos_sims.append(sims)
    return cos_sims


def get_answer(input_test):
    
    with open('vecto.pkl', 'rb') as file:
        vectorizer = pickle.load(file)
    
    data = pd.read_csv('septembre_df.csv', index_col=0)
    data=data[data.Question != ('Bonjour'or'hello'or'yo'or'salut')]
    
    cos_sims = get_cos_sims(input_test)
   
    ind = cos_sims.index(max(cos_sims))

    inds = get_max5(cos_sims)

    
    liste_suggestion=[]
    liste_inter=[]
    liste_sans_doublon=[]
    
    liste_rep=[]
    
    for ix in inds:
        ind_sugg=ix[1]
        
        try:
            questions=data['Question'][data.index[ind_sugg]]        
            liste_inter.append((questions, ind_sugg))
            liste_suggestion.append(questions)  
            reponses = data['Answer'][data.index[ind_sugg]]
            liste_rep.append(reponses)
            
            for  elt in liste_suggestion:
                if elt not in liste_sans_doublon:
                    liste_sans_doublon.append(elt)
            
            continue
        except:
            continue
                 
    return(data['Answer'][data.index[ind]], liste_suggestion, liste_rep, liste_sans_doublon)


def chat():

    print("Bot: Bonjour, bienvenu(e) chez Enercoop! Indiquez q pour quitter le Bot")
    usr = input("Vous: ")
    print("Bot: Comment puis-je vous aider?")

    while True:
        usr = input("Vous: ")
        
        if usr == 'Q' or usr=='q':    
            print("Bot: Merci de votre visite, a bientôt chez Enercoop!.")
            break

        reponse=get_answer(usr)
        reponse=reponse[0]       
        print("Bot:", reponse)        
        print("\n"*2)
        
        outcome = input("Est-ce que cette réponse vous a été utile? Oui/Non: ").lower().strip()            
        if outcome =="merci" or outcome=="oui" or outcome=="au revoir" or outcome=="a plus":
            sugg_choice_0 = input("Bot: Avez-vous une autre question? Oui/Non: ").lower()

            if sugg_choice_0 == 'non':
                print('Parfait! merci à vous et à bientôt sur Enercoop !')                
                break

            else:
                print('Bot: Comment puis-je vous aider ?')

                usr = input("Vous: ")
                reponse=get_answer(usr)
                
                print("Bot:",reponse[0])
                sugg_choice_3 = input("Bot: Avez vous eu la réponse à votre question? Oui/Non: ").lower()

                if sugg_choice_3=='oui':
                    print('Merci de votre visite et à bientôt sur Enercoop!')
                    break


                else:
                    continue
                    
        elif outcome == 'non':            
            cos_sims = get_cos_sims(usr)          
            sugg_choice = input("Bot: Est-ce que je peux me permettre de vous suggérer des questions? Oui/Non: ").lower()

            
            if sugg_choice == 'oui':                
                q_cnt = 1 
                reponse=get_answer(usr)                
                suggestion=reponse[3]
                print(suggestion)
                
                for ind in suggestion:
                    print('-'*50)
                    print(q_cnt, ind)
                    print('-'*50)
                    q_cnt+=1
                    
                print('-'*50)
                print('Question: Autre question?')
                print('-'*50)
                
                liste_possiblibilte = [1,2,3,4,5]
                num = input("Merci d'indiquer le numéro de la question la plus utile ou autre sinon : ")
                
                if(num.lower() =="autre"):
                    print('Merci de reformuler votre question')
                     
                    continue
                
                while(not num.isdigit()): #tant que c'est pas un entier
                    num = input("Merci d'indiquer le numéro de la question la plus utile : ")
                    
                while(int(num) not in liste_possiblibilte):
                    num = input("Merci d'indiquer le numéro de la question la plus utile : ")
      
                num =   int(num)  #REMETTRE NUM EN INT POUR LINDEX                
                rep_choisie=reponse[2]            
                print("Bot", rep_choisie[num-1])                
                
                sugg_choice_2 = input("Bot: Avez vous eu la réponse à votre question? Oui/Non: ").lower()
                
                if sugg_choice_2=='oui':                    
                    print('Merci de votre visite et à bientôt sur Enercoop!')                     
                    break
                    
                else:                    
                    print('Bot: Pourriez vous reformuler votre demande ?')
                    usr = input("Vous: ")                    
                    reponse=get_answer(usr)
                    reponse=reponse[0]                            
                    print("Bot:",reponse) 
                    
                    sugg_choice_3 = input("Bot: Avez vous eu la réponse à votre question? Oui/Non: ").lower()
                
                    if sugg_choice_3=='oui':                    
                        print('Merci de votre visite et à bientôt sur Enercoop!')                     
                        break
                    
                    else:
                        print('Désolé de n avoir pu répondre à votre demain, je vous invite à nous contacter directement par téléphone au 01 75 44 41 56, nous sommes à votre disposition du lundi au vendredi de 9h à 18h.  A bientôt!')
                        break
                    
                    
                    #continue
                
            if outcome == 'non':
                print('Si vous n avez pas eu la réponse à votre question, je vous invite à nous contacter directement par téléphone au 01 75 44 41 56, nous sommes à votre disposition du lundi au vendredi de 9h à 18h.  A bientôt!')
                break




In [4]:
chat()

Bot: Bonjour, bienvenu(e) chez Enercoop! Indiquez q pour quitter le Bot
Vous: yo
Bot: Comment puis-je vous aider?
Vous: je chercjer à déménager


C:\Users\simplon\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\simplon\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Bot: Si vous déménagez, vous devez résilier votre contrat en cours et souscrire un nouveau contrat pour votre nouvelle adresse (nouveau point de livraison, nouvelles spécifications techniques de l'installation, etc...).\n\nDepuis votre espace client, rendez-vous sur l'onglet "Compte", rubrique "Déménagement".\n\nNous vous rappelons que votre espace client est accessible à cette adresse : http://espace-client.enercoop.fr.\n\nVous pouvez également envoyer un mail à l'adresse resiliation@enercoop.org ; en nous transmettant les informations suivantes :\n\nNom et prénom figurant dans votre contrat\n\nAdresse précise du contrat\n\nLa date souhaitée de résiliation\n\nSi possible, le relevé du compteur à la date de sortie (pour avoir une facture de résiliation avec les bons index de consommation)\n\nVotre nouvelle adresse (pour l'envoi de la facture de résiliation)\n\nLa résiliation prendra effet à la date indiquée et au plus tard 30 jours après l'envoi du courriel.



Est-ce que cette réponse

C:\Users\simplon\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\simplon\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Bot: Est-ce que je peux me permettre de vous suggérer des questions? Oui/Non: oui


C:\Users\simplon\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\simplon\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


['Je vais déménager.', 'je déménage, que dois-je faire ?', 'Je souhaite déménager.', 'Déménagement / résiliation', 'Vous déménagez ?\\n']
--------------------------------------------------
1 Je vais déménager.
--------------------------------------------------
--------------------------------------------------
2 je déménage, que dois-je faire ?
--------------------------------------------------
--------------------------------------------------
3 Je souhaite déménager.
--------------------------------------------------
--------------------------------------------------
4 Déménagement / résiliation
--------------------------------------------------
--------------------------------------------------
5 Vous déménagez ?\n
--------------------------------------------------
--------------------------------------------------
Question: Autre question?
--------------------------------------------------
Merci d'indiquer le numéro de la question la plus utile ou autre sinon : 5
Bot Si vous déména